In [3]:
# Importing the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [4]:
# Creating array of sets of random latitude and longitude
lats = np.random.uniform(-90.000,90.000, size = 1500)
lngs = np.random.uniform(-90.000,90.000, size = 1500)
latLngs = zip(lats,lngs)
latLngs

In [5]:
# Adding the latitudes & longitudes to a list
coordinates = list(latLngs)

In [8]:
#Creating list of cities
cities = []

#Identifying the nearest city for each lat and long combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    #Checking to see if the city is unique. If so, it is added to the "cities" list.
    if city not in cities:
        cities.append(city)

#print the city count to confirm there are enough
len(cities)

635